# Chapter 7: Experimental results

## 0. Loading the data and installing libraries

First, we are going to install the dependencies. If you do not have XGBoost on your system please run the following cell:

In [0]:
pip install xgboost

For Google Colab, this cell must be executed each time a notebook is initialized. It will also install ray library.

In [0]:
!pip uninstall -y -q pyarrow
!pip install -q ray

Now we can mount our GDrive:

In [0]:
# Import libraries
import pandas as pd
from google.colab import drive

# Mount GDrive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


After GDrive mounting, we can load the data into a Pandas DataFrame:

In [0]:
# Import this method to convert string of list into a list
from ast import literal_eval

# Load the data into a Pandas DataFrame
review_df = pd.read_csv("/content/drive/My Drive/TFM/yelp_final_data.csv", converters={"text":literal_eval})

# Show 5 columns as an example
review_df.head()

,text,stars
0,"[0.15115709602832794, 1.2984539270401, -0.2926...",1
1,"[-0.5212148427963257, -0.585378885269165, -0.7...",1
2,"[-0.6769105792045593, 1.1047179698944092, 1.45...",1
3,"[-1.2831478118896484, 0.6724963784217834, -0.1...",1
4,"[-1.0676878690719604, 0.5427433848381042, 0.44...",1


For the algorithm to work, stars class should start at 0.

In [0]:
# Substract one star so stars start from 0
review_df['stars'] = review_df['stars']-1

# Show 5 rows as an example
review_df.head()

,text,stars
0,"[0.15115709602832794, 1.2984539270401, -0.2926...",0
1,"[-0.5212148427963257, -0.585378885269165, -0.7...",0
2,"[-0.6769105792045593, 1.1047179698944092, 1.45...",0
3,"[-1.2831478118896484, 0.6724963784217834, -0.1...",0
4,"[-1.0676878690719604, 0.5427433848381042, 0.44...",0


Last, we split the dataset into training and test:

In [0]:
# Import libraries for splitting
import numpy as np
from sklearn.model_selection import train_test_split

# Store input data into an array
X = review_df.text.apply(pd.Series).values
# Store labels into an array
y = review_df.stars.values

# Do a stratifyed split of data. Test data will be a 30% of the total 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2020, stratify=y)

## 1. PBT

First, we need to import the models and metrics. To store the trained models, we will use the os library.

In [0]:
import xgboost as xgb
import sklearn.metrics
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining
import os

We need to define the trainable class for XGBoost. This class will be used for BOHB too.

In [0]:
# We define a trainable class for XGBoost
class XGBoost_trainable(tune.Trainable):

    # Basic setup. Calls XGBoost sklearn classifier
    def _setup(self, config):
        self.model = xgb.XGBClassifier()
        self.params = config

    # Model training    
    def _train(self):
        # Configure hyperparameters for a model
        self.model = self.model.set_params(**self.params)
        # Train a model
        self.model = self.model.fit(X=X_train,y=y_train)
        # Test model's accuracy. Recall that we will use a softprob objective
        # Predict class on test data
        preds = self.model.predict(X_test)
        # Get class of prediction based on its max probability
        pred_labels = np.asarray([np.argmax(line) for line in preds])
        # Compute mean accuracy
        acc = sklearn.metrics.accuracy_score(y_test, pred_labels)
        # Return mean accuracy
        return {"mean_accuracy": acc}

    # Save model
    def _save(self, checkpoint_dir):
        # Create string of directory
        checkpoint_path = os.path.join(checkpoint_dir, "model.pth")
        # Save model using XGBoost method
        self.model.save_model(checkpoint_path)
        return checkpoint_path

    # Restore model
    def _restore(self, checkpoint_path):
        #Restore using XGBoost method
        self.model.load_model(checkpoint_path)

    # Restore config of restored model
    def reset_config(self, new_config):
        self.config = new_config
        return True


We can now execute a PBT search:

In [0]:
# Define number of threads. For Google Colab, the maximum is 2
num_threads = 2
# Define parameters for XGBoost (randomly initialize the studied ones)
config = {
    "verbosity": 0,
    "objective": "multi:softprob",
    "num_class": 5,
    "eval_metric": "merror",
    "max_depth": tune.randint(1, 9),
    "eta": tune.loguniform(1e-4, 1e-1),
    "gamma": tune.loguniform(1e-8, 1.0),
    "grow_policy": tune.choice(["depthwise", "lossguide"])
}

# Create a PBT scheduler
pbt = PopulationBasedTraining(
    metric = "mean_accuracy",
    mode = "max",
    hyperparam_mutations={
      "max_depth": lambda: np.random.randint(1, 9),
      "eta": lambda: np.random.uniform(np.log(1e-4)/np.log(10), np.log(1e-1)/np.log(10)),
      "gamma": lambda: np.random.uniform(np.log(1e-8)/np.log(10), np.log(1)/np.log(10)),
      "grow_policy": ["depthwise", "lossguide"]
    })

# Run PBT and store the outputs to a variable
analysis = tune.run(
    XGBoost_trainable,
    resources_per_trial={"cpu": 1},
    config=config,
    num_samples = 4,
    stop={"training_iteration": 10},
    reuse_actors=True,
    scheduler=pbt)

2020-01-13 13:31:29,625	WARNING worker.py:673 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2020-01-13 13:31:29,629	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-01-13 13:31:29,633	INFO resource_spec.py:216 -- Starting Ray with 6.1 GiB memory available for workers and up to 3.06 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


2020-01-13 13:31:35,063	INFO ray_trial_executor.py:121 -- Trial XGBoost_trainable_027d3ac0: Setting up new remote runner.
2020-01-13 13:31:35,141	WARNING worker.py:1054 -- Warning: The actor XGBoost_trainable has size 24243515 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


Trial name,status,loc,grow_policy,eta,gamma,max_depth
XGBoost_trainable_027d3ac0,RUNNING,,,,,
XGBoost_trainable_02f9e0c0,PENDING,,,,,
XGBoost_trainable_0373cc96,PENDING,,,,,
XGBoost_trainable_03b51f0c,PENDING,,,,,


2020-01-13 13:31:35,450	INFO ray_trial_executor.py:121 -- Trial XGBoost_trainable_02f9e0c0: Setting up new remote runner.
2020-01-13 13:31:35,512	WARNING worker.py:1054 -- Warning: The actor XGBoost_trainable has size 24243515 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


Result for XGBoost_trainable_02f9e0c0:
  date: 2020-01-13_13-33-04
  done: false
  experiment_id: d2f64d6ceaa5461594a0197d30013537
  experiment_tag: 1_eta=0.00056603,gamma=2.9078e-08,grow_policy=lossguide,max_depth=2
  hostname: 9991d65102f3
  iterations_since_restore: 1
  mean_accuracy: 0.2
  node_ip: 172.28.0.2
  pid: 388
  time_since_restore: 87.33367156982422
  time_this_iter_s: 87.33367156982422
  time_total_s: 87.33367156982422
  timestamp: 1578922384
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 02f9e0c0
  


...

Trial name,status,loc,grow_policy,eta,gamma,max_depth,iter,total time (s),acc
XGBoost_trainable_027d3ac0,TERMINATED,,lossguide,0.00704788,-1.46178,0,10,1684.94,0.2
XGBoost_trainable_02f9e0c0,TERMINATED,,lossguide,0.00704788,3.22738e-07,1,10,1529.14,0.2
XGBoost_trainable_0373cc96,TERMINATED,,lossguide,0.00587324,2.68948e-07,1,10,1528.57,0.2
XGBoost_trainable_03b51f0c,RUNNING,172.28.0.2:387,depthwise,0.00106361,0.0116691,4,9,2021.91,0.2


Result for XGBoost_trainable_03b51f0c:
  date: 2020-01-13_15-40-19
  done: true
  experiment_id: cc8379e9a262436f84ca9a1bf757c9b8
  experiment_tag: 3_eta=0.0010636,gamma=0.011669,grow_policy=depthwise,max_depth=4
  hostname: 9991d65102f3
  iterations_since_restore: 3
  mean_accuracy: 0.2
  node_ip: 172.28.0.2
  pid: 387
  time_since_restore: 647.9568338394165
  time_this_iter_s: 202.24020290374756
  time_total_s: 2224.1524727344513
  timestamp: 1578930019
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 03b51f0c
  


Trial name,status,loc,grow_policy,eta,gamma,max_depth,iter,total time (s),acc
XGBoost_trainable_027d3ac0,TERMINATED,,lossguide,0.00704788,-1.46178,0,10,1684.94,0.2
XGBoost_trainable_02f9e0c0,TERMINATED,,lossguide,0.00704788,3.22738e-07,1,10,1529.14,0.2
XGBoost_trainable_0373cc96,TERMINATED,,lossguide,0.00587324,2.68948e-07,1,10,1528.57,0.2
XGBoost_trainable_03b51f0c,TERMINATED,,depthwise,0.00106361,0.0116691,4,10,2224.15,0.2


Trial name,status,loc,grow_policy,eta,gamma,max_depth,iter,total time (s),acc
XGBoost_trainable_027d3ac0,TERMINATED,,lossguide,0.00704788,-1.46178,0,10,1684.94,0.2
XGBoost_trainable_02f9e0c0,TERMINATED,,lossguide,0.00704788,3.22738e-07,1,10,1529.14,0.2
XGBoost_trainable_0373cc96,TERMINATED,,lossguide,0.00587324,2.68948e-07,1,10,1528.57,0.2
XGBoost_trainable_03b51f0c,TERMINATED,,depthwise,0.00106361,0.0116691,4,10,2224.15,0.2


2020-01-13 15:40:19,328	INFO tune.py:334 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.


As we stored the output of our search, we can get some information about PBT's run. For example, we can get best configuration:

In [0]:
print("Best config is", analysis.get_best_config(metric="mean_accuracy"))

Best config is {'verbosity': 0, 'objective': 'multi:softprob', 'num_class': 5, 'booster': 'gbtree', 'eval_metric': 'merror', 'max_depth': 0, 'eta': 0.0070478840420228756, 'gamma': -1.4617833215538054, 'grow_policy': 'lossguide'}


We can also get general data about PBT's run:

In [0]:
analysis.dataframe()

,mean_accuracy,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_this_iter_s,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,trial_id,experiment_tag,config/booster,config/eta,config/eval_metric,config/gamma,config/grow_policy,config/max_depth,config/num_class,config/objective,config/verbosity,logdir
0,0.2,True,NaN,NaN,10,7f6e26f2963b43d4b6f7c8b25c7e722a,2020-01-13_15-29-31,1578929371,335.861557,1684.940611,387,9991d65102f3,172.28.0.2,335.861557,0,1,027d3ac0,"0_eta=0.044127,gamma=0.010735,grow_policy=loss...",gbtree,0.044127,merror,1.073470e-02,lossguide,7,5,multi:softprob,0,/root/ray_results/XGBoost_trainable/XGBoost_tr...
1,0.2,True,NaN,NaN,10,7f6e26f2963b43d4b6f7c8b25c7e722a,2020-01-13_15-28-26,1578929306,180.057062,1529.136116,519,9991d65102f3,172.28.0.2,180.057062,0,1,02f9e0c0,"1_eta=0.00056603,gamma=2.9078e-08,grow_policy=...",gbtree,0.000566,merror,2.907770e-08,lossguide,2,5,multi:softprob,0,/root/ray_results/XGBoost_trainable/XGBoost_tr...
2,0.2,True,NaN,NaN,10,7f6e26f2963b43d4b6f7c8b25c7e722a,2020-01-13_15-31-25,1578929485,179.493554,1528.572608,519,9991d65102f3,172.28.0.2,179.493554,0,1,0373cc96,"2_eta=0.0058732,gamma=2.6895e-07,grow_policy=l...",gbtree,0.005873,merror,2.689480e-07,lossguide,1,5,multi:softprob,0,/root/ray_results/XGBoost_trainable/XGBoost_tr...
3,0.2,True,NaN,NaN,10,cc8379e9a262436f84ca9a1bf757c9b8,2020-01-13_15-40-19,1578930019,202.240203,2224.152473,387,9991d65102f3,172.28.0.2,647.956834,0,3,03b51f0c,"3_eta=0.0010636,gamma=0.011669,grow_policy=dep...",gbtree,0.001064,merror,1.166906e-02,depthwise,4,5,multi:softprob,0,/root/ray_results/XGBoost_trainable/XGBoost_tr...


## 7.2. BOHB

First we need to import BOHB schedulers:

In [0]:
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB

For BOHB, it is required to use the ConfigSpace and hpbandster library. Please run the following cell if you do not have it on your system.

In [0]:
pip install ConfigSpace
pip install hpbandster

We can now import ConfigSpace and define the search space for BOHB

In [0]:
# Import ConfigSpace
import ConfigSpace as CS

# Initiate a configuration space class
config_space = CS.ConfigurationSpace()

# Define the search space
config_space.add_hyperparameter(CS.UniformIntegerHyperparameter("max_depth", lower=1, upper=9))
config_space.add_hyperparameter(CS.UniformFloatHyperparameter("eta", lower=1e-4, upper=1e-1))
config_space.add_hyperparameter(CS.UniformFloatHyperparameter("gamma", lower=1e-8, upper=1.0))
config_space.add_hyperparameter(CS.CategoricalHyperparameter("grow_policy", choices=["depthwise", "lossguide"]));

As with PBT, we need to configure BOHB scheduler:

In [0]:
# Define experiment metrics
experiment_metrics = dict(metric="mean_accuracy", mode="max")

# Define scheduler
bohb_hyperband = HyperBandForBOHB(**experiment_metrics)

# Define search algorithm
bohb_search = TuneBOHB(
    config_space, max_concurrent=4, **experiment_metrics)

Finally, we run a BOHB search:

In [0]:
# Run BOHB search
tune.run(XGBoost_trainable,
    resources_per_trial={"cpu": 1},
    scheduler=bohb_hyperband,
    search_alg=bohb_search,
    reuse_actors=True,
    num_samples=4
    )

2020-01-13 18:21:27,428	INFO ray_trial_executor.py:121 -- Trial XGBoost_trainable_82b61090: Setting up new remote runner.
2020-01-13 18:21:27,476	WARNING worker.py:1054 -- Warning: The actor XGBoost_trainable has size 24243515 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


Trial name,status,loc,grow_policy,eta,gamma,max_depth
XGBoost_trainable_82b61090,RUNNING,,,,,
XGBoost_trainable_82dfcba6,PENDING,,,,,
XGBoost_trainable_83046f9c,PENDING,,,,,
XGBoost_trainable_832a3402,PENDING,,,,,


2020-01-13 18:21:27,793	INFO ray_trial_executor.py:121 -- Trial XGBoost_trainable_82dfcba6: Setting up new remote runner.
2020-01-13 18:21:27,873	WARNING worker.py:1054 -- Warning: The actor XGBoost_trainable has size 24243515 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


Result for XGBoost_trainable_82b61090:
  date: 2020-01-13_18-28-11
  done: false
  experiment_id: 3691cddf60f94eacb008a6ea2a7fe57c
  experiment_tag: 1_eta=0.010571,gamma=0.40645,grow_policy=lossguide,max_depth=8
  hostname: 9991d65102f3
  iterations_since_restore: 1
  mean_accuracy: 0.2
  node_ip: 172.28.0.2
  pid: 4122
  time_since_restore: 400.8107225894928
  time_this_iter_s: 400.8107225894928
  time_total_s: 400.8107225894928
  timestamp: 1578940091
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 82b61090
  


Trial name,status,loc,grow_policy,eta,gamma,max_depth,iter,total time (s),acc
XGBoost_trainable_82b61090,PAUSED,,lossguide,0.0105708,0.406448,8,1,400.811,0.2
XGBoost_trainable_82dfcba6,RUNNING,,,,,,,,
XGBoost_trainable_83046f9c,PENDING,,,,,,,,
XGBoost_trainable_832a3402,PENDING,,,,,,,,


...

Trial name,status,loc,grow_policy,eta,gamma,max_depth,iter,total time (s),acc
XGBoost_trainable_82b61090,TERMINATED,,lossguide,0.0105708,0.406448,8,9,770.978,0.2
XGBoost_trainable_82dfcba6,TERMINATED,,lossguide,0.0556884,0.529778,8,27,3571.55,0.2
XGBoost_trainable_83046f9c,TERMINATED,,depthwise,0.0632253,0.878173,4,27,3114.75,0.2
XGBoost_trainable_832a3402,RUNNING,172.28.0.2:4300,depthwise,0.0314498,0.892913,1,53,1931.53,0.2


Result for XGBoost_trainable_832a3402:
  date: 2020-01-13_20-04-58
  done: true
  experiment_id: dba50441a96149289476f25c4382c3e2
  experiment_tag: 4_eta=0.03145,gamma=0.89291,grow_policy=depthwise,max_depth=1
  hostname: 9991d65102f3
  iterations_since_restore: 27
  mean_accuracy: 0.2
  node_ip: 172.28.0.2
  pid: 4300
  time_since_restore: 705.9188516139984
  time_this_iter_s: 26.05317258834839
  time_total_s: 1957.584547996521
  timestamp: 1578945898
  timesteps_since_restore: 0
  training_iteration: 54
  trial_id: 832a3402
  


Trial name,status,loc,grow_policy,eta,gamma,max_depth,iter,total time (s),acc
XGBoost_trainable_82b61090,TERMINATED,,lossguide,0.0105708,0.406448,8,9,770.978,0.2
XGBoost_trainable_82dfcba6,TERMINATED,,lossguide,0.0556884,0.529778,8,27,3571.55,0.2
XGBoost_trainable_83046f9c,TERMINATED,,depthwise,0.0632253,0.878173,4,27,3114.75,0.2
XGBoost_trainable_832a3402,TERMINATED,,depthwise,0.0314498,0.892913,1,54,1957.58,0.2


Trial name,status,loc,grow_policy,eta,gamma,max_depth,iter,total time (s),acc
XGBoost_trainable_82b61090,TERMINATED,,lossguide,0.0105708,0.406448,8,9,770.978,0.2
XGBoost_trainable_82dfcba6,TERMINATED,,lossguide,0.0556884,0.529778,8,27,3571.55,0.2
XGBoost_trainable_83046f9c,TERMINATED,,depthwise,0.0632253,0.878173,4,27,3114.75,0.2
XGBoost_trainable_832a3402,TERMINATED,,depthwise,0.0314498,0.892913,1,54,1957.58,0.2


2020-01-13 20:04:58,245	INFO tune.py:334 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.
